In [14]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE

set data

In [4]:
data = pd.read_csv("creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
data = data.sample(frac=1)
X = data.iloc[:,:30]
Y = data.iloc[:,30]
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
4725,4158.0,-0.578613,1.177398,2.182287,0.338717,1.300810,1.272276,0.936385,-0.473557,0.983745,...,0.428748,-0.451074,-0.471014,-0.425292,-1.416355,0.181710,-0.528856,-0.427898,-0.558895,5.00
10038,15152.0,0.996978,0.513824,0.595975,2.903621,0.107496,-0.162093,0.328765,-0.210315,0.434882,...,-0.051291,-0.098692,-0.024179,-0.102201,0.379018,0.592869,0.048011,-0.037543,0.020872,75.84
207403,136657.0,-16.458069,-4.929920,-7.727333,-1.282758,-3.954837,1.268130,-0.169124,2.340631,3.442010,...,-7.196235,-1.646318,0.960852,-0.864903,-0.477339,0.705021,-0.399868,-3.007594,1.067970,252.21
211336,138371.0,-1.106864,-2.063825,-3.774108,-0.725220,6.236716,0.824901,-0.373417,0.643320,-0.490578,...,0.614690,0.658780,1.114356,0.632094,0.676298,-0.996001,0.570164,0.078796,0.280175,70.00
99238,67030.0,-0.326023,0.485902,2.510493,1.533488,-0.677684,0.227541,-0.171783,0.108640,0.201271,...,0.062538,0.249164,0.995697,-0.098130,0.522647,-0.554235,-0.150251,-0.059831,-0.074708,0.01


In [11]:
scaler = MinMaxScaler()
for column in X.columns:
  X[f'scaled_{column}'] = scaler.fit_transform(np.array(X[column]).reshape(-1, 1))
  X.drop(columns=[column], inplace=True)
X

,scaled_Time,scaled_V1,scaled_V2,scaled_V3,scaled_V4,scaled_V5,scaled_V6,scaled_V7,scaled_V8,scaled_V9,...,scaled_V20,scaled_V21,scaled_V22,scaled_V23,scaled_V24,scaled_V25,scaled_V26,scaled_V27,scaled_V28,scaled_Amount
4725,0.024064,0.948464,0.779682,0.875230,0.266945,0.774473,0.275811,0.271060,0.780306,0.496668,...,0.584830,0.554208,0.488058,0.659118,0.191381,0.588106,0.339061,0.408613,0.301782,1.946195e-04
10038,0.087689,0.975231,0.772680,0.847741,0.380645,0.766440,0.261390,0.267358,0.783129,0.477761,...,0.579719,0.559889,0.508903,0.663916,0.433307,0.611186,0.433291,0.415818,0.313547,2.951988e-03
207403,0.790876,0.678692,0.715240,0.703510,0.195067,0.739092,0.275770,0.264325,0.810493,0.581351,...,0.503643,0.534940,0.554855,0.652589,0.317913,0.617481,0.360131,0.360998,0.334796,9.816995e-03
211336,0.800795,0.939490,0.745482,0.772014,0.219782,0.807702,0.271313,0.263081,0.792286,0.445880,...,0.586810,0.572099,0.562016,0.674821,0.473365,0.521999,0.518583,0.417965,0.318809,2.724673e-03
99238,0.387923,0.952755,0.772385,0.880917,0.319908,0.761154,0.265307,0.264309,0.786551,0.469713,...,0.580931,0.565496,0.556480,0.663976,0.452661,0.546796,0.400905,0.415407,0.311608,3.892389e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122044,0.442231,0.951236,0.773574,0.861820,0.144863,0.769139,0.258020,0.271356,0.781990,0.466865,...,0.582463,0.558686,0.497663,0.658920,0.322799,0.602384,0.410749,0.414106,0.309525,1.623126e-04
138624,0.478998,0.979663,0.771212,0.842782,0.282486,0.763390,0.252320,0.265987,0.783046,0.462559,...,0.579574,0.556881,0.472274,0.667305,0.428979,0.590580,0.440792,0.416118,0.313758,5.021182e-05
139217,0.480722,0.741958,0.861281,0.717544,0.281512,0.726075,0.379727,0.133945,0.244536,0.259953,...,0.689577,0.192494,0.848186,0.708132,0.169242,0.497359,0.367029,0.420362,0.313543,1.584203e-02
99144,0.387668,0.979271,0.765333,0.843029,0.259661,0.765222,0.271073,0.261356,0.789626,0.478226,...,0.578489,0.559623,0.491569,0.664854,0.196502,0.588345,0.477027,0.416408,0.313033,1.942302e-04


clustering

In [21]:
kmeans = KMeans(n_clusters=2, init='k-means++')
kmeans.fit(X)
predicted_labels = kmeans.labels_

conf_matrix = confusion_matrix(data['Class'], predicted_labels)

class_report = classification_report(data['Class'], predicted_labels)
print("Classification Report:")
print(class_report)



c:\Users\Parsa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.54      0.70    284315
           1       0.00      0.32      0.00       492

    accuracy                           0.54    284807
   macro avg       0.50      0.43      0.35    284807
weighted avg       1.00      0.54      0.70    284807



classification

In [17]:
from sklearn import svm

over sampling

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42,stratify=Y)
smote = SMOTE(random_state=156, sampling_strategy=0.05)
X_train, Y_train = smote.fit_resample(X_train, Y_train)
X_train

,scaled_Time,scaled_V1,scaled_V2,scaled_V3,scaled_V4,scaled_V5,scaled_V6,scaled_V7,scaled_V8,scaled_V9,...,scaled_V20,scaled_V21,scaled_V22,scaled_V23,scaled_V24,scaled_V25,scaled_V26,scaled_V27,scaled_V28,scaled_Amount
0,0.911998,0.994236,0.757610,0.828471,0.235716,0.758143,0.256863,0.260270,0.783860,0.476505,...,0.573531,0.558668,0.525516,0.667476,0.384256,0.574237,0.462661,0.417140,0.312166,0.000195
1,0.654579,0.993537,0.766715,0.809176,0.249446,0.769430,0.260042,0.265897,0.783816,0.478064,...,0.578519,0.566141,0.555484,0.664423,0.413618,0.600836,0.353233,0.416561,0.311885,0.000039
2,0.373646,0.980579,0.770478,0.832246,0.265665,0.769373,0.265138,0.265441,0.785335,0.464404,...,0.579904,0.555901,0.466574,0.665128,0.199727,0.597408,0.459623,0.416417,0.313415,0.000077
3,0.256291,0.872327,0.698101,0.873175,0.346835,0.805123,0.214657,0.237847,0.794026,0.507866,...,0.595111,0.573793,0.523092,0.682924,0.422911,0.564337,0.347400,0.415896,0.319789,0.000039
4,0.118258,0.859721,0.812889,0.719195,0.355058,0.743093,0.238476,0.252728,0.826110,0.497008,...,0.580467,0.558231,0.472165,0.663679,0.428573,0.576992,0.353436,0.416838,0.308130,0.003503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238818,0.588068,0.507852,0.976780,0.387241,0.752724,0.649712,0.308665,0.030134,0.445435,0.061454,...,0.672468,0.266086,0.805194,0.724494,0.388575,0.528731,0.393741,0.321919,0.304069,0.000068
238819,0.499735,0.922168,0.783047,0.814889,0.293747,0.762999,0.269558,0.245655,0.766324,0.483667,...,0.584486,0.562632,0.545226,0.659222,0.343453,0.589318,0.396226,0.423193,0.315300,0.001277
238820,0.891735,0.948549,0.749746,0.740382,0.386137,0.758096,0.248184,0.262758,0.786097,0.432349,...,0.602664,0.576905,0.489551,0.651041,0.316187,0.579305,0.440997,0.421779,0.321044,0.037430
238821,0.257359,0.822768,0.835879,0.639488,0.438440,0.721613,0.230806,0.220801,0.845999,0.377368,...,0.589026,0.576282,0.477352,0.666632,0.374979,0.601787,0.381996,0.438228,0.318165,0.003892


SVM

In [22]:
clf = svm.SVC(kernel="rbf").fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
class_report = classification_report(Y_test, Y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.79      0.86      0.82        98

    accuracy                           1.00     56962
   macro avg       0.90      0.93      0.91     56962
weighted avg       1.00      1.00      1.00     56962

